In [1]:
import time
import pandas as pd
from tqdm import tqdm

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

In [3]:
def df_market_influence(driver):
    df = pd.DataFrame(columns=['발표날짜', '발표'])
    time.sleep(10)
    for i in range(100, 0, -1):  # 반복 범위 수정
        try:
            data_xpath = f'/html/body/div/div/div/div/div/div/div/div/div/div/div/div/div/table/tbody/tr[{i}]/td[1]'
            data_element = WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.XPATH, data_xpath)))
            data_parts = data_element.text.split('.')[:2]
            formatted_data = '.'.join(data_parts)
            index_xpath = f'/html/body/div/div/div/div/div/div/div/div/div/div/div/div/div/table/tbody/tr[{i}]/td[2]'
            index_element = WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.XPATH, index_xpath)))
            index_text = index_element.text
            df.loc[100-i] = [formatted_data, index_text]
        except Exception as e:
            pass
    return df

In [4]:
def find_economic_indicator(driver, save_info_dict):
    time.sleep(10)
    for i in range(1, 30):
        try:
            market_influence_xpath = f'/html/body/div/div/div/div/div/div/div/div/div/div/ul/li[{i}]/div/div/div[2]/dl/dd[3]'
            market_influence = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.XPATH, market_influence_xpath))
            )

            if market_influence.text in ["높음", "매우 높음"]:
                save_info_list = []
                economic_indicator_xpath = f'/html/body/div/div/div/div/div/div/div/div/div/div/ul/li[{i}]/div/div/div/a'
                economic_indicator_name = WebDriverWait(driver, 1).until(
                    EC.element_to_be_clickable((By.XPATH, economic_indicator_xpath))
                )
                save_info_list.append(market_influence.text)
                print(economic_indicator_name.text)
                economic_text = economic_indicator_name.text
                economic_indicator_name.click()

                market_data = df_market_influence(driver)
                save_info_list.append(market_data)
                
                # 시트의 첫 번째 열에 키 값을 넣어줌
                save_info_dict[economic_text] = save_info_list
                driver.back()
                time.sleep(10)
        except:
            pass

In [5]:
def scrape_economic_indicators(nation):
    driver = webdriver.Chrome()
    driver.get('https://www.naver.com/')
    save_info_dict = {}  # save_info_dict 초기화
    
    search_xpath = '/html/body/div[2]/div[1]/div/div[3]/div/div/form/fieldset/div/input'
    search = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, search_xpath))
            )
    search.send_keys(f"{nation} 경제지표")
    search.send_keys(Keys.RETURN)
    for i in range(1, 21):
        try:
            large_category_xpath = f'/html/body/div/div/div/div/div/div/div/div/div/div/div/div/ul/li[{i}]/a/span'
            Large_category = WebDriverWait(driver, 1).until(
                EC.element_to_be_clickable((By.XPATH, large_category_xpath))
            )
            
            driver.execute_script("arguments[0].click();", Large_category)
            
            find_economic_indicator(driver, save_info_dict)  # save_info_dict를 인자로 전달
        except Exception as e:
            try:
                next_button_xpath = '/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div/div[2]/a[2]'
                next_button = WebDriverWait(driver, 1).until(
                    EC.element_to_be_clickable((By.XPATH, next_button_xpath))
                )
                
                driver.execute_script("arguments[0].click();", next_button)
                
                Large_category = WebDriverWait(driver, 1).until(
                    EC.element_to_be_clickable((By.XPATH, large_category_xpath))
                )
                
                driver.execute_script("arguments[0].click();", Large_category)
                
                find_economic_indicator(driver, save_info_dict)  # save_info_dict를 인자로 전달
            except:
                pass

    driver.quit()

    with pd.ExcelWriter(f'{nation} economic_indicators.xlsx') as writer:
        for key, value in save_info_dict.items():
            # 시트의 첫 번째 열에 키 값을 씀
            df = value[1]
            df.index.name = key
            # Excel 파일에 쓰기
            df.to_excel(writer, sheet_name=key)

In [6]:
# scrape_economic_indicators('미국')

In [8]:
# '미국','한국','중국', '일본', '유로', '인도','대만','홍콩','러시아', '영국', '프랑스', '사우디아라비아', '이스라엘', '캐나다', '호주'
nation_list = ['독일']
for nation in tqdm(nation_list, desc="Scraping economic indicators"):
    scrape_economic_indicators(nation)

Scraping economic indicators:   0%|                                                              | 0/1 [00:00<?, ?it/s]

독일 국내총생산(GDP) 전분기 대비
독일 국내총생산(GDP) 전년대비
독일 생산자물가지수(PPI) 전월대비
독일 소비자물가지수(CPI) 전월대비
독일 소비자물가조화지수(HICP) 전년대비
독일 무역수지
독일 Ifo 기업체감지수
독일 기대평가지수
독일 현행평가지수
독일 ZEW 경기동향지수
독일 ZEW 경제전망지수
독일 실업률
독일 실업 변동
독일 산업생산 전년대비
독일 산업생산 전월대비
독일 공장수주 전월대비


Scraping economic indicators: 100%|████████████████████████████████████████████████████| 1/1 [17:12<00:00, 1032.26s/it]
